# TA Session 2: Exercises
## Identification, ARMA Models, and Wold Decomposition

**Econometrics II - NYU PhD Program**

This notebook contains computational exercises for TA Session 2. We will:
1. Verify stationarity and invertibility conditions
2. Compute autocovariances and ACFs
3. Simulate ARMA processes
4. Visualize ACF patterns


In [ ]:
# Import necessary libraries
import numpy as np
import matplotlib.pyplot as plt
from scipy import signal
from statsmodels.tsa.arima_process import ArmaProcess
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf
from statsmodels.tsa.stattools import acf, pacf
import warnings
warnings.filterwarnings('ignore')

# Set plot style
plt.style.use('seaborn-v0_8-whitegrid')
plt.rcParams['figure.figsize'] = (12, 4)
plt.rcParams['font.size'] = 12


---
## Part 1: Checking Stationarity and Invertibility

### Helper Functions


In [ ]:
def check_stationarity(ar_coeffs):
    """
    Check if AR process is stationary by examining roots of characteristic polynomial.
    
    Parameters:
    -----------
    ar_coeffs : list
        AR coefficients [phi_1, phi_2, ..., phi_p]
        For AR(p): y_t = phi_1*y_{t-1} + ... + phi_p*y_{t-p} + u_t
    
    Returns:
    --------
    dict with roots, their moduli, and stationarity status
    """
    # Characteristic polynomial: 1 - phi_1*z - phi_2*z^2 - ... - phi_p*z^p
    poly_coeffs = [1] + [-c for c in ar_coeffs]
    roots = np.roots(poly_coeffs[::-1])
    moduli = np.abs(roots)
    
    is_stationary = all(m > 1 for m in moduli)
    
    return {
        'roots': roots,
        'moduli': moduli,
        'is_stationary': is_stationary
    }

def check_invertibility(ma_coeffs):
    """
    Check if MA process is invertible by examining roots of MA polynomial.
    
    Parameters:
    -----------
    ma_coeffs : list
        MA coefficients [theta_1, theta_2, ..., theta_q]
        For MA(q): y_t = u_t + theta_1*u_{t-1} + ... + theta_q*u_{t-q}
    
    Returns:
    --------
    dict with roots, their moduli, and invertibility status
    """
    # MA polynomial: 1 + theta_1*z + theta_2*z^2 + ... + theta_q*z^q
    poly_coeffs = [1] + list(ma_coeffs)
    roots = np.roots(poly_coeffs[::-1])
    moduli = np.abs(roots)
    
    is_invertible = all(m > 1 for m in moduli)
    
    return {
        'roots': roots,
        'moduli': moduli,
        'is_invertible': is_invertible
    }


### Example: AR(2) Stationarity Check


In [ ]:
# Example: AR(2) with phi_1 = 1.5, phi_2 = -0.56
ar_coeffs = [1.5, -0.56]
result = check_stationarity(ar_coeffs)

print("AR(2) Process: y_t = 1.5*y_{t-1} - 0.56*y_{t-2} + u_t")
print("="*50)
print(f"Roots of characteristic polynomial: {result['roots']}")
print(f"Moduli of roots: {result['moduli']}")
print(f"Is stationary: {result['is_stationary']}")
print()

# Visualize roots in complex plane
fig, ax = plt.subplots(figsize=(6, 6))
theta = np.linspace(0, 2*np.pi, 100)
ax.plot(np.cos(theta), np.sin(theta), 'k--', label='Unit Circle')
ax.scatter(result['roots'].real, result['roots'].imag, s=100, c='red', marker='x', 
           linewidths=3, label='Roots')
ax.axhline(y=0, color='gray', linewidth=0.5)
ax.axvline(x=0, color='gray', linewidth=0.5)
ax.set_xlim(-2, 2)
ax.set_ylim(-2, 2)
ax.set_aspect('equal')
ax.set_xlabel('Real')
ax.set_ylabel('Imaginary')
ax.set_title('Roots of AR(2) Characteristic Polynomial\n(Stationarity: roots outside unit circle)')
ax.legend()
plt.tight_layout()
plt.show()


### Example: MA(1) Observational Equivalence

Demonstrating that $\theta$ and $1/\theta$ give the same autocorrelation at lag 1.


In [ ]:
# Demonstrate that theta and 1/theta give same autocorrelation at lag 1
def ma1_rho1(theta):
    """Compute rho_1 for MA(1) process"""
    return theta / (1 + theta**2)

theta_values = np.linspace(-3, 3, 1000)
theta_values = theta_values[theta_values != 0]  # Remove zero

rho1_values = ma1_rho1(theta_values)

fig, ax = plt.subplots(figsize=(10, 5))
ax.plot(theta_values, rho1_values, 'b-', linewidth=2)
ax.axhline(y=0, color='gray', linewidth=0.5)
ax.axvline(x=0, color='gray', linewidth=0.5)
ax.axvline(x=1, color='red', linestyle='--', label='θ = 1')
ax.axvline(x=-1, color='red', linestyle='--', label='θ = -1')

# Highlight the observational equivalence
theta_example = 0.5
theta_equiv = 1/theta_example
rho_value = ma1_rho1(theta_example)
ax.scatter([theta_example, theta_equiv], [rho_value, rho_value], 
           s=100, c='green', zorder=5, label=f'θ={theta_example} and θ={theta_equiv}')
ax.axhline(y=rho_value, color='green', linestyle=':', alpha=0.5)

ax.set_xlabel('θ')
ax.set_ylabel('ρ₁')
ax.set_title('MA(1): Observational Equivalence\nρ₁ = θ/(1+θ²) is same for θ and 1/θ')
ax.legend()
ax.set_xlim(-3, 3)
ax.set_ylim(-0.6, 0.6)
plt.tight_layout()
plt.show()

print(f"For θ = {theta_example}: ρ₁ = {ma1_rho1(theta_example):.4f}")
print(f"For θ = {theta_equiv}: ρ₁ = {ma1_rho1(theta_equiv):.4f}")
print(f"\nBoth give the same ρ₁, but only |θ| < 1 is invertible!")


---
## Part 2: ACF Patterns for AR, MA, ARMA

Comparing the characteristic ACF patterns of different process types.


In [ ]:
def compute_ar_acf(phi, nlags=20):
    """Compute theoretical ACF for AR(1) process"""
    return [phi**k for k in range(nlags+1)]

def compute_ma_acf(theta_list, sigma2=1, nlags=20):
    """Compute theoretical ACF for MA(q) process"""
    q = len(theta_list)
    theta = [1] + list(theta_list)  # Include theta_0 = 1
    
    # Variance
    gamma_0 = sigma2 * sum([t**2 for t in theta])
    
    # Autocovariances
    acf_values = [1]  # rho_0 = 1
    
    for k in range(1, nlags+1):
        if k > q:
            acf_values.append(0)
        else:
            gamma_k = sigma2 * sum([theta[j] * theta[j+k] for j in range(q+1-k)])
            acf_values.append(gamma_k / gamma_0)
    
    return acf_values

# Compare ACF patterns
fig, axes = plt.subplots(1, 3, figsize=(15, 4))
nlags = 15

# AR(1) with phi = 0.8
phi = 0.8
ar_acf = compute_ar_acf(phi, nlags)
axes[0].stem(range(nlags+1), ar_acf, basefmt='k-')
axes[0].set_xlabel('Lag')
axes[0].set_ylabel('ACF')
axes[0].set_title(f'AR(1) with φ = {phi}\n(Geometric decay)')
axes[0].set_ylim(-0.2, 1.1)

# MA(2) with theta_1 = 0.5, theta_2 = 0.3
theta_list = [0.5, 0.3]
ma_acf = compute_ma_acf(theta_list, nlags=nlags)
axes[1].stem(range(nlags+1), ma_acf, basefmt='k-')
axes[1].set_xlabel('Lag')
axes[1].set_ylabel('ACF')
axes[1].set_title(f'MA(2) with θ₁={theta_list[0]}, θ₂={theta_list[1]}\n(Cuts off after lag 2)')
axes[1].set_ylim(-0.2, 1.1)

# ARMA(1,1) - simulated
np.random.seed(42)
ar = np.array([1, -0.7])  # AR polynomial: 1 - 0.7L
ma = np.array([1, 0.4])   # MA polynomial: 1 + 0.4L
arma_process = ArmaProcess(ar, ma)
sample = arma_process.generate_sample(nsample=5000)
arma_acf = acf(sample, nlags=nlags)
axes[2].stem(range(nlags+1), arma_acf, basefmt='k-')
axes[2].set_xlabel('Lag')
axes[2].set_ylabel('ACF')
axes[2].set_title(f'ARMA(1,1) with φ=0.7, θ=0.4\n(Geometric decay)')
axes[2].set_ylim(-0.2, 1.1)

plt.tight_layout()
plt.show()


---
## Part 3: Common Factor Problem

Demonstrating that ARMA(1,1) with φ = -θ is equivalent to white noise.


In [ ]:
# ARMA(1,1) with common factor
phi = 0.6
theta = -phi  # Common factor: phi = -theta

print(f"ARMA(1,1) with φ = {phi}, θ = {theta}")
print(f"This means: (1 - {phi}L)y_t = (1 + {theta}L)u_t")
print(f"            (1 - {phi}L)y_t = (1 - {phi}L)u_t")
print(f"            y_t = u_t  (White Noise!)")
print()

# Simulate and compare ACFs
np.random.seed(42)
n = 2000

# Pure white noise
wn = np.random.randn(n)

# ARMA(1,1) with common factor (should look like white noise)
ar = np.array([1, -phi])
ma = np.array([1, theta])
arma_process = ArmaProcess(ar, ma)
arma_cf = arma_process.generate_sample(nsample=n)

# Compare ACFs
fig, axes = plt.subplots(1, 2, figsize=(12, 4))

plot_acf(wn, ax=axes[0], lags=20, title='White Noise ACF')
plot_acf(arma_cf, ax=axes[1], lags=20, title=f'ARMA(1,1) with φ={phi}, θ={theta}\n(Common Factor)')

plt.tight_layout()
plt.show()

print("\nNotice: Both ACFs show no significant autocorrelation beyond lag 0!")
print("The ARMA(1,1) with φ = -θ is observationally equivalent to white noise.")


---
## Part 4: Exercise Solutions (Placeholders)

The following sections contain placeholders for the specific exercises from:
- **Tim's Practice Problems** (Exercises 2, 3)
- **Medeiros ARMA Exercises** (1, 2, 3, 6)
- **Medeiros Vector Processes** (Exercise 4)

Solutions will be added once the problem statements are provided.

### Tim's Exercise 2


In [ ]:
# PLACEHOLDER: Tim's Exercise 2
# Problem statement: [To be added from practice_problems_week2.pdf]
# Solution:
print("Tim's Exercise 2")
print("="*50)
print("Problem statement: [To be added]")
print()
print("Solution approach:")
print("  Step 1: [To be completed]")
print("  Step 2: [To be completed]")
print("  Step 3: [To be completed]")


### Tim's Exercise 3


In [ ]:
# PLACEHOLDER: Tim's Exercise 3
print("Tim's Exercise 3")
print("="*50)
print("Problem statement: [To be added]")
print()
print("Solution: [To be completed]")


### Medeiros ARMA Exercises (1, 2, 3, 6)

From ECO2LISTA.PDF - translated from Portuguese.


In [ ]:
# PLACEHOLDER: Medeiros ARMA Exercises
exercises = [1, 2, 3, 6]

for ex in exercises:
    print(f"Medeiros ARMA Exercise {ex}")
    print("="*50)
    print("Problema (Traduzido): [A ser inserido]")
    print("Solução: [A completar]")
    print()


### Medeiros Vector Processes Exercise 4


In [ ]:
# PLACEHOLDER: Medeiros Vector Processes Exercise 4
print("Medeiros Vector Processes Exercise 4")
print("="*50)
print("Problema (Traduzido): [A ser inserido]")
print("Solução: [A completar]")
print()
print("This exercise may involve VAR processes and multivariate analysis.")


---
## Summary

This notebook covered:

1. **Stationarity checks**: Using root analysis for AR processes
2. **Invertibility checks**: Using root analysis for MA processes  
3. **Observational equivalence**: MA(1) example with θ and 1/θ
4. **ACF patterns**: Geometric decay (AR) vs cutoff (MA)
5. **Common factor problem**: ARMA(1,1) reducing to white noise

**Note**: Exercise solutions are placeholders. Update this notebook once the problem statements from the PDFs are provided.
